# Equal Lethargy Flux Spectrum

The beauty of creating our inputs in Python is that we have access to the full power of a real programming language and all its third-party packages. In this exercise, you are asked to determine (and plot) the flux spectrum in 1000 equal lethargy-width groups covering $10^{-5}$ eV to 20 MeV. The model is already set up for you, so you just need to create an appropriate tally, run the model, and retrieve the data from the statepoint file.

HINT: You may want to use `math.log10`, `numpy.logspace`, and `matplotlib.pyplot.loglog`.

In [ ]:
%matplotlib inline
from math import log10
import matplotlib.pyplot as plt
import numpy as np
import openmc

import sys
sys.path.append('..')
from inputs import *

In [ ]:
pincell = PinCellInputSet()
pincell.export()